In [81]:
import pandas as pd

In [87]:
# load openai api key
import os
os.environ['OPENAI_API_KEY'] = 'YOUR KEY HERE'

# train the model
# !openai api fine_tunes.create -t train_prepared_train.jsonl -v train_prepared_valid.jsonl -m davinci --compute_classification_metrics --classification_n_classes 2 --classification_positive_class sarc

In [96]:
# check status
!openai api fine_tunes.follow -i ft-wBRSkwg1jHjfp3gthRabBZv7

[2023-07-24 00:59:43] Created fine-tune: ft-wBRSkwg1jHjfp3gthRabBZv7
[2023-07-24 03:52:43] Fine-tune costs $0.62
[2023-07-24 03:52:43] Fine-tune enqueued. Queue number: 3
[2023-07-24 03:56:22] Fine-tune is in the queue. Queue number: 2
[2023-07-24 03:56:25] Fine-tune is in the queue. Queue number: 1
[2023-07-24 03:56:50] Fine-tune is in the queue. Queue number: 0
[2023-07-24 03:56:51] Fine-tune started
[2023-07-24 04:00:56] Completed epoch 1/4
[2023-07-24 04:05:11] Completed epoch 2/4
[2023-07-24 04:09:23] Completed epoch 3/4
[2023-07-24 04:13:34] Completed epoch 4/4
[2023-07-24 04:14:25] Uploaded model: babbage:ft-university-of-florida-2023-07-24-08-14-24
[2023-07-24 04:14:26] Uploaded result file: file-P2r7tiiKjIpPWVxFQBAnxYhL
[2023-07-24 04:14:26] Fine-tune succeeded

Job complete! Status: succeeded 🎉
Try out your fine-tuned model:

openai api completions.create -m babbage:ft-university-of-florida-2023-07-24-08-14-24 -p <YOUR_PROMPT>


In [98]:
!openai api fine_tunes.results -i ft-wBRSkwg1jHjfp3gthRabBZv7 > finetune_results.csv

In [99]:
fine_tune_results = pd.read_csv('finetune_results.csv')
fine_tune_results

,step,elapsed_tokens,elapsed_examples,training_loss,training_sequence_accuracy,training_token_accuracy,validation_loss,validation_sequence_accuracy,validation_token_accuracy,classification/accuracy,classification/precision,classification/recall,classification/auroc,classification/auprc,classification/f1.0
0,1,1096,8,0.234884,0.0,0.000000,0.213988,0.000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2128,16,0.239317,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2840,24,0.270923,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,4256,32,0.128478,0.0,0.071429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,5608,40,0.112064,0.0,0.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2104,2105,2248520,16840,0.013740,1.0,1.000000,0.021301,0.875,0.916667,NaN,NaN,NaN,NaN,NaN,NaN
2105,2106,2249488,16848,0.018352,1.0,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2106,2107,2250776,16856,0.010223,1.0,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2107,2108,2251488,16864,0.017210,1.0,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [100]:
# prepare list of prompts
test_dataset = pd.read_json('test_prepared.jsonl', lines=True)

print( 'No. of test records: ', len(test_dataset) )

No. of test records:  1304


In [101]:
# initialize the list of answers
results = []

In [102]:
# get predictions
import time
from tqdm import tqdm
from openai.error import ServiceUnavailableError


for index in tqdm( range(len(test_dataset)), desc='Running requests' ):
    max_retries = 3  # Set the maximum number of retries
    num_retries = 0

    while num_retries < max_retries:
        try:
            completion = openai.Completion.create(
                model="babbage:ft-university-of-florida-2023-07-24-08-14-24", 
                prompt=test_dataset['prompt'][index],
                max_tokens=1,
            )

            result = completion.get('choices')[0].get('text')
            if result == ' sarc' or result == ' not':
                results.append({'text': test_dataset['prompt'][index], 'prediction': result})
            else:
                print(result)
                time.sleep(2) # Wait for 1 second before making another request
                continue

            time.sleep(2) # Wait for 1 second before making another request
            break  # Break out of the while loop if no error occurred
        
        except ServiceUnavailableError as e:
            # Handle the OpenAIError and retry the request
            num_retries += 1
            print(f"Error occurred from OpenAI, retrying request ({num_retries}/{max_retries}) for batch index {index}: {e}")
            
            time.sleep(num_retries)  # Wait for an incremental time before retrying


    if num_retries == max_retries:
        results.append({'text': test_dataset['prompt'][index], 'prediction': 'max_retries'})
        print(f"Max retries reached for request {index}. Skipping to the next one.")

Running requests: 100%|██████████| 1304/1304 [47:06<00:00,  2.17s/it] 


In [103]:
# print predictions
print(len(results))
print(results)

1304
[{'text': "If that's true, then Freedom of Speech is doomed. Harassment is subjective. Now I can claim that a book I don't like is harassing me, and it will be banned.\n\n###\n\n", 'prediction': ' sarc'}, {'text': "So, IOW, you are saying that those of us who are intelligent enough to realize that your non existent imaginary god doesn't exist have a chemical imbalance? Wow, that's quite a stretch even for you Archie baby.\n\n###\n\n", 'prediction': ' sarc'}, {'text': 'And I see you still flip back and fourth from being an atheist to not being an atheist.\n\n###\n\n', 'prediction': ' sarc'}, {'text': 'Really, so you are totally knowable? Show me the site that has the Mach User-Manual where I can know everything about you.\n\n###\n\n', 'prediction': ' sarc'}, {'text': "I don't see anyone without healthcare. None of my friends, none of my family, none of my neighbors. ZILCH. NADA. NAUGHT. CIPHER.   Now I don't go wandering around the inner cities looking for half-crazy winos and crac

In [113]:
# compare results
accuracy = 0
problematic_sentences = []
y_true = test_dataset['completion'].to_list()
y_pred = [result['prediction'] for result in results]

for i in range(len(y_true)):
    if ( ('not' in y_true[i]) and ('not' in y_pred[i]) ) or ( ('not' not in y_true[i]) and ('not' not in y_pred[i]) ):
        accuracy += 1
    else:
        problematic_sentences.append({
            'sentence': test_dataset['prompt'][i],
            'true': y_true[i],
            'pred': y_pred[i]
        })

accuracy = round( accuracy / len(y_true), 2 )
print('Accuracy = ', accuracy)

Accuracy =  0.82


In [114]:
results_df = pd.DataFrame(results)
results_df.to_csv('single_training_results.csv', index=False)

In [115]:
problematic_sentences_df = pd.DataFrame(problematic_sentences)
problematic_sentences_df.to_csv('single_training_problematic_sentences.csv', index=False)